# **Main Notebook for Thesis Project**

In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
# filespace
from data import Data

In [2]:
# agent for accessing API
AGENT = {'User-Agent': 'mehmet.ozturk@kcl.ac.uk'}
# getting CIK DataFrame
ciks = requests.get(
    'https://www.sec.gov/files/company_tickers.json',
    headers = AGENT
)
# success check
if ciks.status_code == 200:
    print(f'Status: [{ciks.status_code}], CIK data obtained successfully.')
else:
    print(f'Status: [{ciks.status_code}], error otaining CIK.')

Status: [200], CIK data obtained successfully.


In [3]:
# transforming into DataFrame
ciks = pd.DataFrame.from_dict(
    ciks.json(),
    orient = 'index'
)
# formatting to string
ciks = ciks.astype(str)
ciks.head()

,cik_str,ticker,title
0,789019,MSFT,MICROSOFT CORP
1,320193,AAPL,Apple Inc.
2,1045810,NVDA,NVIDIA CORP
3,1652044,GOOGL,Alphabet Inc.
4,1018724,AMZN,AMAZON COM INC


In [4]:
AAPL = Data('AAPL')
AAPL.get_cik(ciks)
AAPL.get_metadata()
AAPL.process_filings()


Respone: [200], metadata obtained successfully.
Respone: [200], filing obtained successfully.
Current filename:	 a10-qq220183312018.htm
OIE page found
OIE table found
BS page found
BS table found
CF page found
CF table found
Respone: [200], filing obtained successfully.
Current filename:	 a10-qq320186302018.htm
OIE page found
OIE table found
BS page found
BS table found
CF page found
CF table found


In [5]:
AAPL.data

{'2018-03-31': {'revenue': 61137,
  'gross profit': 23422,
  'operating income': 15894,
  'net income': 13822,
  'eps': 2.75,
  'current assets': 130053,
  'current liabilities': 89320,
  'shareholder equity': 126878,
  'net cash operating': 43423,
  'net cash investing': 15120,
  'net cash financing': 33773,
  'cash': 45059},
 '2018-06-30': {'revenue': 53265,
  'gross profit': 20421,
  'operating income': 12612,
  'net income': 11519,
  'eps': 2.36,
  'current assets': 115761,
  'current liabilities': 88548,
  'shareholder equity': 114949,
  'net cash operating': 57911,
  'net cash investing': 19067,
  'net cash financing': 65296,
  'cash': 31971}}